### Second attempt at training
This time im trying to set up an environments that would allow me to have a variety of bots for generating the training data

In [1]:
from TensorRTS import Agent, GameResult, Interactive_TensorRTS

from tournament_runner import Bot, Matchup, Bracket, Tournament
import os
import sys
import os
# tensor_path = os.path.abspath(os.path.join(os.path.basename(__file__), os.pardir, os.pardir, os.pardir))
# sys.path.append(tensor_path)

import random 
from typing import Dict, List, Mapping, Tuple, Set
from entity_gym.env import Observation
from entity_gym.runner import CliRunner
from entity_gym.env import *
from TensorRTS import Agent
import torch
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")


In [2]:
#Rush agent on the left side
class Rush_Agent_Player_2(Agent):
    def __init__(self, init_observation : Observation, action_space : Dict[ActionName, ActionSpace]) -> None: 
        super().__init__(init_observation, action_space)

    def take_turn(self, current_game_state : Observation) -> Mapping[ActionName, Action]:
        mapping = {}
        if current_game_state.features['Tensor'][1][2] > 0 :
            #rush
            mapping['Move'] = GlobalCategoricalAction(0, self.action_space['Move'].index_to_label[1])
        else:
        #     #advance
            mapping["Move"] = GlobalCategoricalAction(1, self.action_space['Move'].index_to_label[2])
        return mapping
    
    def on_game_start(self) -> None:
        return super().on_game_start()
    
    def on_game_over(self, did_i_win : bool, did_i_tie : bool) -> None:
        return super().on_game_over(did_i_win, did_i_tie)

In [3]:
#Rush agent on the right side
class Rush_Agent_Player_1(Agent):
    def __init__(self, init_observation : Observation, action_space : Dict[ActionName, ActionSpace]) -> None: 
        super().__init__(init_observation, action_space)

    def take_turn(self, current_game_state : Observation) -> Mapping[ActionName, Action]:
        mapping = {}
        if current_game_state.features['Tensor'][0][2] > 0 :
            #rush
            mapping['Move'] = GlobalCategoricalAction(0, self.action_space['Move'].index_to_label[0])
        else:
        #     #advance
            mapping["Move"] = GlobalCategoricalAction(1, self.action_space['Move'].index_to_label[2])
        return mapping
    
    def on_game_start(self) -> None:
        return super().on_game_start()
    
    def on_game_over(self, did_i_win : bool, did_i_tie : bool) -> None:
        return super().on_game_over(did_i_win, did_i_tie)

In [4]:
#includes rush and boom
class Random_Agent(Agent):
    def __init__(self, init_observation : Observation, action_space : Dict[ActionName, ActionSpace]) -> None: 
        super().__init__(init_observation, action_space)

    def take_turn(self, current_game_state : Observation) -> Mapping[ActionName, Action]:
        mapping = {}

        action_choice = random.randrange(0, 4)
        mapping['Move'] = GlobalCategoricalAction(0, self.action_space['Move'].index_to_label[action_choice])
        return mapping
    
    def on_game_start(self) -> None:
        return super().on_game_start()
    
    def on_game_over(self, did_i_win : bool, did_i_tie : bool) -> None:
        return super().on_game_over(did_i_win, did_i_tie)

In [5]:
#some useful functions
import functools
def flatten_dict_of_arrays(data, prefix=""):
  flattened_list = []
  def flatten_inner(value, key):
    if isinstance(value, list):
      for i, v in enumerate(value):
        flatten_inner(v, f"{prefix}{key}[{i}]")
    else:
      flattened_list.append((f"{prefix}{key}", value))

  for key, value in data.items():
    flatten_inner(value, key)

  return [value for _, value in flattened_list]


def value_to_discrete_4(input):
    output = [0,0,0,0]
    output[input] = 1
    return output

index_to_moves = {
    0 : "advance",
    1 : "retreat",
    2 : "rush",
    3 : "boom"
}
moves_to_index = {
    "advance": 0,
    "retreat": 1,
    "rush": 2,
    "boom": 3,
}

In [6]:
#rewritten gameRunner to save game state sequence
class GameRunnerSaveStates(): 
    def __init__(self, environment = None, enable_printouts : bool = False):
        self.game = Interactive_TensorRTS(enable_printouts=enable_printouts)
        self.game.reset()

        self.player_one = None
        self.player_two = None
        self.results : GameResult = None
        self.observations = []
        self.actions = []
        self.rewards = []
        self.dones = []
    
    def assign_players(self, first_agent : Agent, second_agent : Agent = None):
        self.player_one = first_agent

        if second_agent is not None:
            self.player_two = second_agent

    def run(self): 
        assert(self.player_one is not None)

        game_state = self.game.observe()
        self.player_one.on_game_start()
        if self.player_two is not None: 
            self.player_two.on_game_start()

        while(self.game.is_game_over is False):
            #take moves and pass updated environments to agents

            self.observations.append(flatten_dict_of_arrays(game_state.features))
            action = self.player_one.take_turn(game_state)
            self.actions.append(value_to_discrete_4(action["Move"].index))

            game_state = self.game.act(action)
            self.peepee = game_state

            self.rewards.append(game_state.reward)
            self.dones.append(game_state.done)
            if (self.game.is_game_over is False):
                if self.player_two is None: 
                    game_state = self.game.opponent_act()
                else:
                    #future player_two code
                    game_state = self.game.act(self.player_two.take_turn(game_state), False, True)

        # who won? 
        tie = False
        win_p_one = False
        win_p_two = False

        p_one = self.game.tensor_power(0)
        p_two = self.game.tensor_power(1)

        if p_one > p_two: 
            win_p_one = True
        elif p_two > p_one:
            win_p_two = True
        else:
            tie = True

        self.results = GameResult(win_p_one, win_p_two, tie)
        self.player_one.on_game_over(win_p_one, tie)
        if self.player_two is not None:
            self.player_two.on_game_over(win_p_two, tie)

In [7]:
def generate_dataset(num_runs,agent1,agent2):
    datasetDict = {
        'observations': [],
        'actions': [], 
        'rewards': [], 
        'dones': [],
    }

    #generates training data
    for _ in range(num_runs):
        game_runner = GameRunnerSaveStates(enable_printouts=False)

        observation = game_runner.game.observe()
        action_space = game_runner.game.action_space()

        player1 = agent1(observation,action_space)
        player2 = agent2(observation,action_space)

        game_runner.assign_players(player1, player2)
        game_runner.run()

        datasetDict['observations'].append(game_runner.observations)
        datasetDict['actions'].append(game_runner.actions)
        datasetDict['rewards'].append(game_runner.rewards)
        datasetDict['dones'].append(game_runner.dones)

    from datasets import Dataset
    dataset = Dataset.from_dict(datasetDict)
    return dataset



### Training on two rush bots

In [8]:
rushDataset = generate_dataset(1000,Rush_Agent_Player_1,Rush_Agent_Player_2)


In [56]:
rushDataset[2]['observations']

[[3, 5, 8, 8, 12, 1, 19, 1, 23, 8, 28, 5, 13, 1, 2, 0, 18, 1, 2, 0],
 [3, 5, 8, 8, 12, 1, 19, 1, 23, 8, 28, 5, 14, 1, 2, 0, 17, 1, 2, 0],
 [3, 5, 8, 8, 12, 1, 19, 1, 23, 8, 28, 5, 15, 1, 2, 0, 16, 1, 2, 0]]

In [58]:
from transformers import DecisionTransformerConfig, Trainer, TrainingArguments

import DTscripts

collator = DTscripts.DecisionTransformerGymDataCollator(rushDataset)
config = DecisionTransformerConfig(state_dim=collator.state_dim, act_dim=collator.act_dim)
model = DTscripts.TrainableDT(config)

os.environ["WANDB_DISABLED"] = "true" 
training_args = TrainingArguments(
    output_dir="output/",
    remove_unused_columns=False,
    num_train_epochs=500,
    per_device_train_batch_size=64,
    learning_rate=1e-4,
    weight_decay=1e-4,
    warmup_ratio=0.1,
    optim="adamw_torch",
    max_grad_norm=0.25,
    use_mps_device=True,
    # evaluation_strategy="steps",
    # eval_steps = 15 #needs a evaluation dataset 

)
training_args = training_args.set_save(strategy="steps", steps=500)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=rushDataset,
    data_collator=collator,
)

trainer.train()
trainer.save_model("./output/rush")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


  0%|          | 0/8000 [00:00<?, ?it/s]

{'loss': 0.0629, 'learning_rate': 6.25e-05, 'epoch': 31.25}
{'loss': 0.0004, 'learning_rate': 9.722222222222223e-05, 'epoch': 62.5}
{'loss': 0.0002, 'learning_rate': 9.027777777777779e-05, 'epoch': 93.75}
{'loss': 0.0001, 'learning_rate': 8.333333333333334e-05, 'epoch': 125.0}
{'loss': 0.0, 'learning_rate': 7.638888888888889e-05, 'epoch': 156.25}
{'loss': 0.0, 'learning_rate': 6.944444444444444e-05, 'epoch': 187.5}
{'loss': 0.0, 'learning_rate': 6.25e-05, 'epoch': 218.75}
{'loss': 0.0, 'learning_rate': 5.555555555555556e-05, 'epoch': 250.0}
{'loss': 0.0, 'learning_rate': 4.8611111111111115e-05, 'epoch': 281.25}
{'loss': 0.0, 'learning_rate': 4.166666666666667e-05, 'epoch': 312.5}
{'loss': 0.0, 'learning_rate': 3.472222222222222e-05, 'epoch': 343.75}
{'loss': 0.0, 'learning_rate': 2.777777777777778e-05, 'epoch': 375.0}
{'loss': 0.0, 'learning_rate': 2.0833333333333336e-05, 'epoch': 406.25}
{'loss': 0.0, 'learning_rate': 1.388888888888889e-05, 'epoch': 437.5}
{'loss': 0.0, 'learning_rate

### Evaluation

In [126]:
from TensorRTS import TensorRTS


def get_move_no_lookback(model,state):
    state_dim = 20
    act_dim = 4
    states = torch.from_numpy(state).reshape(1, 1, state_dim).to(device=device, dtype=torch.float32)
    actions = torch.zeros((1, 1, act_dim), device=device, dtype=torch.float32)
    rewards = torch.zeros(1, 1, device=device, dtype=torch.float32)
    target_return = torch.tensor(10, dtype=torch.float32).reshape(1, 1)
    timesteps = torch.tensor(0, device=device, dtype=torch.long).reshape(1, 1)
    attention_mask = torch.zeros(1, 1, device=device, dtype=torch.float32)


    with torch.no_grad():
        state_preds, action_preds, return_preds = model(
            states=states,
            actions=actions,
            rewards=rewards,
            returns_to_go=target_return,
            timesteps=timesteps,
            attention_mask=attention_mask,
            return_dict=False,
            # use_mps_device=True

        )
    return state_preds, action_preds, return_preds

def evaluate_model(model,num_runs):
    testEnv = TensorRTS(enable_prinouts = False)
    possible_moves = {
        0 : "advance",
        1 : "retreat",
        2 : "rush",
        3 : "boom"
    }
    wins = 0
    losses = 0
    ties = 0
    for _ in range(num_runs):

        testEnv.reset()
        for _ in range(1000):
            action = {}
            #make prediction
            model_out = get_move_no_lookback(model,np.array(flatten_dict_of_arrays(testEnv.observe().features)))
            # print(model_out[1])
            # if possible_moves[torch.argmax(model_out[1]).item()] == "advance" : print("advance")
            # print(torch.argmax(model_out[1]).item())
            action['Move'] = GlobalCategoricalAction(0, possible_moves[torch.argmax(model_out[1]).item()])
            pred_reward = model_out[2]
            result = testEnv.act(action)
            # print(model_out[1])

            if result.done == True: 
                break 
            

        # print(result.reward)
        if (result.reward == 10): 
            wins+=1
        if (result.reward == 0): ties +=1
        if (result.reward == -10): losses+=1


    return ((wins+ties)/num_runs)

In [128]:
from transformers import DecisionTransformerModel

model = DecisionTransformerModel.from_pretrained("./output/rush")

evaluate_model(model,100) #its not good :(

0.22